## Word Count
Counting the number of occurances of words in a text is one of the most popular first eercises when learning Map-Reduce Programming. It is the equivalent to `Hello World!` in regular programming.

We will do it two way, a simpler way where sorting is done after the RDD is collected, and a more sparky way, where the sorting is also done using an RDD.

In [7]:
#start the SparkContext
!pip install findspark
from pyspark import SparkContext
sc = SparkContext(master="local[3]")

ImportError: No module named pyspark

## Read text into an RDD

### Download data file from S3

In [4]:
%%time
import urllib
data_dir='../../Data'
filename='Moby-Dick.txt'
f = urllib.urlretrieve ("https://mas-dse-open.s3.amazonaws.com/"+filename, data_dir+'/'+filename)

# First, check that the text file is where we expect it to be
!ls -l $data_dir/$filename

-rw-r--r-- 1 root root 1257260 Apr 16  2017 ../../Data/Moby-Dick.txt
CPU times: user 220 ms, sys: 640 ms, total: 860 ms
Wall time: 15.2 s


### Define an RDD that will read the file
Note that, as execution is Lazy, this does not necessarily mean that actual reading of the file content has occured.

In [5]:
%%time
text_file = sc.textFile(data_dir+'/'+filename)
type(text_file)

NameError: name 'sc' is not defined

## Counting the words
* split line by spaces.
* map `word` to `(word,1)`
* count the number of occurances of each word.

In [ ]:
%%time
counts = text_file.flatMap(lambda line: line.split(" ")) \
             .filter(lambda x: x!='')\
             .map(lambda word: (word, 1)) \
             .reduceByKey(lambda a, b: a + b)
type(counts)

### Have a look a the execution plan
Note that the earliest node in the dependency graph is the file `../../Data/Moby-Dick.txt`. 

In [ ]:
print counts.toDebugString()

### Count!
Finally we count the number of times each word has occured.
Now, finally, the Lazy execution model finally performs some actual work, which takes a significant amount of time.

In [ ]:
%%time
Count=counts.count()
Sum=counts.map(lambda (w,i): i).reduce(lambda x,y:x+y)
print 'Count=%f, sum=%f, mean=%f'%(Count,Sum,float(Sum)/Count)

## Finding the most common words
* `counts`: RDD with 33301 pairs of the form `(word,count)`. 
* Find the 2 most frequent words. 
* **Method1:** `collect` and sort on head node.
* **Method2:** Pure Spark, `collect` only at the end.

### Method1: `collect` and sort on head node 
#### Collect the RDD into the driver node
* Collect can take significant time.

In [ ]:
%%time
C=counts.collect()
print type(C)

#### Sort 
* RDD collected into list in driver node.
* No longer using spark parallelism.
* Sort in python
* will not scale to very large documents.

In [ ]:
C.sort(key=lambda x:x[1])
print 'most common words\n','\n'.join(['%s:\t%d'%c for c in C[-5:]])
print '\nLeast common words\n','\n'.join(['%s:\t%d'%c for c in C[:5]])

### Compute the mean number of occurances per word.

In [ ]:
Count2=len(C)
Sum2=sum([i for w,i in C])
print 'count2=%f, sum2=%f, mean2=%f'%(Count2,Sum2,float(Sum2)/Count2)


### **Method2:** Pure Spark, `collect` only at the end.
* Collect into the head node only the more frquent words.
* Requires multiple **stages**

#### Step 1 split, clean and map to `(word,1)`

In [ ]:
%%time
RDD=text_file.flatMap(lambda x: x.split(' '))\
    .filter(lambda x: x!='')\
    .map(lambda word: (word,1))
    

#### **Step 2** Count occurances of each word.

In [ ]:
%%time
RDD1=RDD.reduceByKey(lambda x,y:x+y)

#### **Step 3** Reverse `(word,count)` to `(count,word)` and sort by key

In [ ]:
%%time
RDD2=RDD1.map(lambda (c,v):(v,c))
RDD3=RDD2.sortByKey(False)

#### Full execution plan

We now have a complete plan to compute the most common words in the text. Nothing has been executed yet! Not even one one bye has been read from the file `Moby-Dick.txt` !

For more on execution plans and lineage see [jace Klaskowski's blog](https://jaceklaskowski.gitbooks.io/mastering-apache-spark/content/spark-rdd-lineage.html#toDebugString)

In [ ]:
print 'RDD3:'
print RDD3.toDebugString()

#### **Step 4** Take the top 5 words. **only now the computer executes the plan!**

In [ ]:
%%time
C=RDD3.take(5)
print 'most common words\n','\n'.join(['%d:\t%s'%c for c in C])